In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
#数据预处理方式
def prepare_data(x,y):
    x = tf.cast(x,dtype=tf.float32)/255.
    y = tf.one_hot(y,depth=10)
    y=tf.cast(y,dtype=tf.int64)
    return x,y

In [3]:
#载入数据和预处理
(x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
train_db = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(128).map(prepare_data).shuffle(10000)
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(128).map(prepare_data).shuffle(10000)
train_iter = iter(train_db)
sample = next(train_iter)
print(sample[0].shape,sample[1].shape)

(128, 28, 28) (128, 10)


In [11]:
#初始化系数
w1 = tf.Variable(tf.random.truncated_normal((784,256),stddev=0.1))
b1 = tf.Variable(tf.zeros(256))
w2 = tf.Variable(tf.random.truncated_normal((256,128),stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
w3 = tf.Variable(tf.random.truncated_normal((128,10),stddev=0.1))
b3 = tf.Variable(tf.zeros(10))
lr = 1e-3

In [13]:
for epoch in range(10):
    for step,(x,y) in enumerate(train_db):
        x = tf.reshape(x,[-1,28*28])
        with tf.GradientTape() as tape:
            out1 = tf.nn.relu(x@w1 + b1)
            out2 = tf.nn.relu(out1@w2+b2)
            out3 = tf.cast(out2@w3+b3,dtype=tf.int64)
            loss=tf.reduce_mean(tf.square(y-out3))
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        print(grads)
        w1.assign_sub(lr * grads[0])    #原地更新,数据类型保持不变,
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])
        if step % 100 == 0:
            print(epoch, step, 'loss:', float(loss))
        total_correct, total_number = 0, 0
    for step, (x,y) in enumerate(test_db):

        # x_test [b, 28,28] => [b, 28*28]
        x_test = tf.reshape(x, [-1, 28*28])

        # [b, 784] => [b, 256] => [b, 128] => [b, 10]
        h1 = tf.nn.relu(x_test@w1 + b1)
        h2 = tf.nn.relu(h1@w2 + b2)
        out = h2@w3 + b3

        # out: [b, 10] 这里的out属于实数的范围R。
        # prob: [b, 10]实数范围映射到[0, 1]范围内。
        prob = tf.nn.softmax(out, axis=1)  #是在[b, 10]中的10维度上面。故axis=1
        # 预测值：选择概率最大的值所在的位置。[b, 10] ==> [b]
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        # 真实值：y: [b] 这里我们可以发现做测试的时候，编码方式不需要转换为one-hot,只需要把索引和pred比较。
        # [b] int32类型。
        # print(pred.dtype, y.dtype) 运行结果：<dtype: 'int64'> <dtype: 'int32'> 类型不匹配
        correct = tf.cast(tf.equal(pred, y),dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_correct +=int(correct)        #总的正确的个数，
        total_number +=x.shape[0]           #总的测试的个数。


    #循环结束以后：
    acc = total_correct /total_number
    print("test acc: ", acc)

[None, None, None, None, None, None]


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import os


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'   #屏蔽无关的信息，2是只打印error
#自动查看有没有缓存一个mnist数据集，没有的话自动从google下载。
# x: [60k, 28, 28],    x_test: [10k, 28, 28]
# y: [60k]             y_test: [10k]
(x,y),(x_test,y_test) =datasets.mnist.load_data()

# 把数据类型转换为张量
#x: [0~255] => [0~ 1.]
x=tf.convert_to_tensor(x,dtype=tf.float32) / 255.
y=tf.convert_to_tensor(y,dtype=tf.int32)

x_test=tf.convert_to_tensor(x_test,dtype=tf.float32) / 255.
y_test=tf.convert_to_tensor(y_test,dtype=tf.int32)

print(x.shape, y.shape, x.dtype, y.dtype)
print(tf.reduce_min(x),tf.reduce_max(x))
print(tf.reduce_min(y),tf.reduce_max(y))


#创建一个数据集，一次取到一个batch_size,因为上面一次取一个。
train_db = tf.data.Dataset.from_tensor_slices((x,y)).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(128)

# train_iter = iter(train_db)   #迭代器
# sample = next(train_iter)
# print("batch: ", sample[0].shape, sample[1].shape)

#创建权值，完成前向传播
#[batch,784] => [b, 256] => [b, 128] => [b, 10]
#w: [dim_in, dim_out]
#b: [dim_out]
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))#原来均值为0，方差维为1,现在方差变为0.1
b1 = tf.Variable(tf.zeros([256]))
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

lr = 1e-3

#前向运算


for epoch in range(100):         #iterate 迭代整个数据集10次。
    # 外层for: 对所有的图片做循环。
    #for (x, y) in train_db:
    for step, (x, y) in enumerate(train_db):  #迭代for every batch
        #x: [128, 28, 28]
        #y: [128]

        #维度变换; [b, 28, 28]-> [b, 28*28]
        x = tf.reshape(x, [-1, 28*28])
        #x: [b, 28*28]
        #h1 = x@w1 + b1
        #[b, 784]@[784, 256] + [256] = [b, 256] + [256] => [b, 256] + [b, 256]

        #使用tensorflow自动求导的过程,其中: w,b。tf.GradientTape()参与梯度计算的代码放到这里面。
        with tf.GradientTape() as tape:
            #GradientTape里面默认只会跟踪tf.Variable()类型。如果类型不是这个的话。这里为tf.tensor,tf.Variable是tf.tensor的一种特殊类型。
            #因此简单的包装一下。

            h1 = x@w1 + tf.broadcast_to(b1, [x.shape[0], 256]) #直接自动广播机制，也可以手动
            h1=tf.nn.relu(h1)
            #[b, 256] -> [b, 128]
            h2 = h1@w2 + b2
            h2=tf.nn.relu(h2)
            #[b, 128] -> [b, 10]
            out = h2@w3 + b3  #得到前向输出结果。

            #computer loss: 计算误差均方差。
            # out维度: [b, 10]
            # 真实的y:  [b],维度这里需要把y变成一个one-hot 编码
            y_onehot = tf.one_hot(y, depth=10)

            #mse = mean((y_onehot-out)^2)
            #shape : [b, 10]
            loss = tf.square(y_onehot - out)

            #mean: scalar
            #loss = tf.reduce_mean(loss) / b / 10 一般来说除以一个b就够了，每个batch上的一个均值。取决于怎么理解，都是可以的。
            loss = tf.reduce_mean(loss) #这里相当于一个放缩，正向放缩不会影响梯度的方向。没有影响的。
            #loss = tf.sum(tf.reduce_mean(loss,axis=1)) / b,我自己的理解。

        #得到一个梯度。需要求解梯度的有哪些呢？
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        #print(grads)  #结果：[None, None, None, None, None, None]
        # w1 = w1 - lr* w1_grad, 这里为了细节，手写。实际可以不用手写。

        #w1 = w1 - lr * grads[0]         #这里两个w1为两个对象。原来的w1减去这个值赋值给一个新的对象.w1原来是
        							    #tf.Variable,更新一次之后新的w1变为tf.Tensor类型了。新一次操作之后就会错误。
        #b1 = b1 - lr * grads[1]
        #w2 = w2 - lr * grads[2]
        #b2 = b2 - lr * grads[3]
        #w3 = w3 - lr * grads[4]
        #b3 = b3 - lr * grads[5]   #第一for增加一个step
        w1.assign_sub(lr * grads[0])    #原地更新,数据类型保持不变,
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])

        # print(isinstance(b3,tf.Variable))
        # print(isinstance(b3,tf.Tensor))
        #每100论，看一下loss信息。
        if step % 100 == 0:
            print(epoch, step, 'loss:', float(loss))


    # test/evluation做测试
    #注意这里必须使用当前的(最新的) [w1, b1, w2, b2, w3, b3]
    total_correct, total_number = 0, 0
    for step, (x,y) in enumerate(test_db):

        # x_test [b, 28,28] => [b, 28*28]
        x_test = tf.reshape(x, [-1, 28*28])

        # [b, 784] => [b, 256] => [b, 128] => [b, 10]
        h1 = tf.nn.relu(x_test@w1 + b1)
        h2 = tf.nn.relu(h1@w2 + b2)
        out = h2@w3 + b3

        # out: [b, 10] 这里的out属于实数的范围R。
        # prob: [b, 10]实数范围映射到[0, 1]范围内。
        prob = tf.nn.softmax(out, axis=1)  #是在[b, 10]中的10维度上面。故axis=1
        # 预测值：选择概率最大的值所在的位置。[b, 10] ==> [b]
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        # 真实值：y: [b] 这里我们可以发现做测试的时候，编码方式不需要转换为one-hot,只需要把索引和pred比较。
        # [b] int32类型。
        # print(pred.dtype, y.dtype) 运行结果：<dtype: 'int64'> <dtype: 'int32'> 类型不匹配
        correct = tf.cast(tf.equal(pred, y),dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_correct +=int(correct)        #总的正确的个数，
        total_number +=x.shape[0]           #总的测试的个数。


    #循环结束以后：
    acc = total_correct /total_number
    print("test acc: ", acc)


(60000, 28, 28) (60000,) <dtype: 'float32'> <dtype: 'int32'>
tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0, shape=(), dtype=int32) tf.Tensor(9, shape=(), dtype=int32)
0 0 loss: 0.35197287797927856
0 100 loss: 0.18710939586162567
0 200 loss: 0.18492546677589417
0 300 loss: 0.15776900947093964
0 400 loss: 0.13826946914196014
test acc:  0.1716
1 0 loss: 0.1342138946056366
1 100 loss: 0.14012067019939423
1 200 loss: 0.151187464594841
1 300 loss: 0.1359231024980545
1 400 loss: 0.12222625315189362
test acc:  0.2272
2 0 loss: 0.11679490655660629
2 100 loss: 0.12546411156654358
2 200 loss: 0.13325053453445435
2 300 loss: 0.12204279005527496
2 400 loss: 0.1116311177611351
test acc:  0.2759
3 0 loss: 0.10520273447036743
3 100 loss: 0.11521227657794952
3 200 loss: 0.12084205448627472
3 300 loss: 0.11205847561359406
3 400 loss: 0.10401581227779388
test acc:  0.3207
4 0 loss: 0.09673832356929779
4 100 loss: 0.10757486522197723
4 200 loss: 0.111599959433